## Job scheduling with Airflow

In this part of the tutorial, you'll learn how to automate running jobs in Hopsworks with the Airflow integration. This can be useful if you have tasks that need to be repeated with a certain time interval, such as ingesting and preprocessing data or training a new model.

- Create a job
- Create an Airflow DAG (graph that describes how steps in the workflow are connected)

## Creating a job

You can create a job in the Hopsworks UI as described in the [documentation](https://docs.hopsworks.ai/hopsworks/2.5.0/compute/jobs/). A job can be a JAR file, a Python script or a Jupyter Notebook

(TODO: Check if this is still the case as the new documentation suggests is is more flexible now)

In this example, we'll use a Jupyter notebook.

There are four types of job you can create: Spark, Flink, Python and Docker, where the latter two are only available in the Enterprise Edition of Hopsworks. In this example, we will use a Spark job.

Let's try to schedule a daily retraining of our fraud model. Of course, in a real project, this would also require that new data have been continually ingested and preprocessed. Those steps could also be automated in a similar way?

(TODO: Check if there is documentation about creating a job programmatically)

In the new Hopsworks UI, go to `Jobs => New Job`, give the job a name (maybe "retrain" for instance), click `From Project` and navigate to the `4_model_training_and_registration.ipynb` file.